In [ ]:
import torch
import torch.nn as nn
import numpy
import pandas
import matplotlib.pyplot as plt
import torch.utils.data as tdata
import os 
from torch import dtype
from google.colab import drive
import skimage
from PIL import Image
import itertools
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
drive.mount('/content/drive', force_remount=True)


def rescale_resnet(model):
    first_conv_layer = [nn.Conv2d(1, 3, 4, stride=1, padding=1, dilation=1, groups=1, bias=True)]
    first_conv_layer.extend([model.features])  
    model.features= nn.Sequential(*first_conv_layer )
    return model
    

model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
model_conv.fc = torch.nn.Linear(model_conv.fc.in_features, 9)#says it doesnt recognize model_conv.classifier
model_conv = nn.Sequential(nn.Conv2d(1, 3, 4, stride=1, padding=1, dilation=1, groups=1, bias=True),model_conv)
model_conv_2= torchvision.models.resnet18(pretrained=True)
model_conv_2.fc = torch.nn.Linear(model_conv_2.fc.in_features,3)
for param in model_conv_2.parameters():
    param.requires_grad = False
model_conv_2 = nn.Sequential(nn.Conv2d(1, 3, 4, stride=1, padding=1, dilation=1, groups=1, bias=True),model_conv_2)

model_conv = model_conv.to(device)
model_conv_2 = model_conv_2.to(device)


    

#we need a custom grayscale resnet to train
class GrayScale_Resnet(nn.Module):
    def __init__(self, num_classes):
        super(GrayScale_Resnet,self).__init__()


#hyperparameters for CNNs
btch_sz = 64
learning_rate = 0.3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epoch = 12
MODEL_NOTE_PATH = '/content/drive/My Drive/data/Current-Model-Note/model_one.txt'
MODEL_ACC_PATH = '/content/drive/My Drive/data/Current-Model-Acc/model_two.txt'

#NOTE TO SELF: REVISE LABEL TXT FILE BECAUSE THE FLAT/SHARP/NATS ARE MISPLACED (ALSO DEAL WITH DYNAMIC LATER, 
#MORE PICTURES NEEDED)

def process_labels_note(label_set):
    for i in open(label_set).readlines():
        strn = i
        #print(strn)
        #print("{},{}".format(ord(strn[0]), ord('A')))
        char_arr= strn.split(',')
        #for j in char_arr:
            #print("{} + {}".format("CHAR ELEM:", j))
        tensor_conv = [0 for i in range(7)]
        tensor_dynamic = [0 for i in range(2)]
        tensor_conv[ord(char_arr[0]) - ord('A')]=1
        #print("{}:{}".format("STRN 1",strn[1]))


        if(char_arr[1]=='F'):
            tensor_dynamic[0] = 1
        elif(char_arr[1]=='P'):
            tensor_dynamic[1] = 1
        vectorized_note = []
        vectorized_note.append(tensor_conv + tensor_dynamic)
    
        yield vectorized_note
def process_labels_acc(label_set):
    for i in open(label_set).readlines():
        tensor_acc = [0 for k in range(3)]
        #print(i)
        i=i.replace('\n','')
        if(i == 'FLAT'):
            #print(0)
            tensor_acc[0] = 1
        elif(i == 'NAT'):
            #print(1)
            tensor_acc[1] = 1
        elif(i == 'SHARP'):
            #print(2)
            tensor_acc[2] = 1
        
        yield tensor_acc

class Gesture_Dataset_Note_and_Volume(tdata.Dataset):
    @staticmethod
    def run():
        pass
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        for lbl in range(len(self.labels)):
            self.labels[lbl] = torch.tensor(self.labels[lbl]).squeeze(0)
        #self.images = torch.tensor(self.images)
        #self.labels= torch.tensor(self.labels)
    def __getitem__(self,idx):
        return self.images[idx],self.labels[idx]
    def __len__(self):
        return len(self.images)

class Gesture_Dataset_Accidental(tdata.Dataset):
    def __init__(self,images,labels,transform=None):
        self.images=images
        self.labels=labels
        for lbl in range(len(self.labels)):
            self.labels[lbl] = torch.tensor(self.labels[lbl]).squeeze(0)
        #self.images= torch.tensor(self.images)
        #self.labels= torch.tensor(self.labels)
    def __getitem__(self,idx):
        return self.images[idx],self.labels[idx]
    def __len__(self):
        return len(self.images)

#pitch will be based on the angle at which the left hand is oriented
#volume will be based on the fingers' separation from each other

class CNN_Base_Note_and_Volume(nn.Module):
    #assume 
    def __init__(self, input_size_x, input_size_y):
        super(CNN_Base_Note_and_Volume, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1,3,12)
        self.relu_1 = torch.nn.ReLU()
        self.mp_1 = torch.nn.MaxPool2d(5)
        self.conv2d_2 = torch.nn.Conv2d(3,6,9)
        self.relu_2 = torch.nn.ReLU()
        self.mp_2 = torch.nn.MaxPool2d(5)
        self.conv2d_3 = torch.nn.Conv2d(6,9,6)
        self.relu_3 = torch.nn.ReLU()
        self.mp_3 = torch.nn.MaxPool2d(3)
        self.conv2d_4=torch.nn.Conv2d(9,12,4)
        self.relu_4 = torch.nn.ReLU()
        self.mp_4 = torch.nn.MaxPool2d(2)
        self.conv2d_5 = torch.nn.Conv2d(12,14,3)
        self.relu_5 = torch.nn.ReLU()
        self.mp_5 = torch.nn.MaxPool2d(2)
        self.ftn_1 = torch.nn.Linear(720, 300)
        self.ftn_2= torch.nn.Linear(300, 100)
        self.ftn_3=torch.nn.Linear(100, 9)
        #fully connected layer
    def forward(self, input_note_image):
        one_conv = self.conv2d_1(input_note_image)
        one_rel = self.relu_1(one_conv)
        one_mp = self.mp_1(one_rel)
        two_conv = self.conv2d_2(one_mp)
        two_rel = self.relu_2(two_conv)
        two_mp = self.mp_2(two_rel)
        three_conv = self.conv2d_3(two_mp)
        three_rel = self.relu_3(three_conv)
        three_mp = self.mp_3(three_rel)
        four_conv = self.conv2d_4(three_mp)
        four_rel = self.relu_4(four_conv)
        inp_linear = self.ftn_1(three_mp)
        inp_two = self.ftn_2(inp_linear)
        inp_three = self.ftn_3(inp_two)
        return inp_three
        #print(input_image.shape)
        
#accidental will be based on the extending of some finger
class CNN_Base_Accidental(nn.Module):
    #consider using less conv/relu layers for accidental, may lead to overfitting
    def __init__(self, input_x, input_y):
        super(CNN_Base_Accidental, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1,3,12)
        self.relu_1 = torch.nn.ReLU()
        self.mp_1 = torch.nn.MaxPool2d(5)
        self.conv2d_2 = torch.nn.Conv2d(3,6,11)
        self.relu_2 = torch.nn.ReLU()
        self.mp_2 = torch.nn.MaxPool2d(5)
        self.conv2d_3 = torch.nn.Conv2d(6,9,9)
        self.relu_3 = torch.nn.ReLU()
        self.mp_3 = torch.nn.MaxPool2d(3)
        self.conv2d_4=torch.nn.Conv2d(9,12,5)
        self.relu_4 = torch.nn.ReLU()
        self.mp_4 = torch.nn.MaxPool2d(2)
        self.conv2d_5 = torch.nn.Conv2d(12,14,3)
        self.relu_5 = torch.nn.ReLU()
        self.mp_5 = torch.nn.MaxPool2d(2)
        self.ftn_1 = torch.nn.Linear(720,300)
        self.ftn_2= torch.nn.Linear(300,100)
        self.ftn_3=torch.nn.Linear(100,11)
    def forward(self, input_acc_image):
        one_conv = self.conv2d_1(input_acc_image)
        one_rel = self.relu_1(one_conv)
        one_mp = self.mp_1(one_rel)
        two_conv = self.conv2d_2(one_mp)
        two_rel = self.relu_2(two_conv)
        two_mp = self.mp_2(two_rel)
        three_conv = self.conv2d_3(two_mp)
        three_rel = self.relu_3(three_conv)
        three_mp = self.mp_3(three_rel)
        four_conv = self.conv2d_4(three_mp)
        four_rel = self.relu_4(four_conv)
        inp_linear = self.ftn_1(three_mp)
        inp_two = self.ftn_2(inp_linear)
        inp_three = self.ftn_3(inp_two)
        return inp_three

train_set_note = os.listdir("/content/drive/My Drive/data/Image_Training_Note")
test_set_note = os.listdir("/content/drive/My Drive/data/Image_Testing_Note")
train_set_accidental = os.listdir('/content/drive/My Drive/data/Image_Training_Accidental')
test_set_accidental = os.listdir('/content/drive/My Drive/data/Image_Testing_Accidental')
label_set = os.path.join("/content/drive/My Drive/data/Labels/tonal_labels_train_and_test_note.txt")
label_set_two = os.path.join("/content/drive/My Drive/data/Labels/tonal_labels_train_and_test_accidental.txt")
image_train= []
image_acc = []
image_test=[]
labels_note = []
labels_accidental = []

Gesture_Dataset_Note_and_Volume.run()
'''
transform_img = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
'''
print("NOTE/DYNAMIC IMAGE TRAIN SET")
for i in range(1,len(train_set_note)):
    print(train_set_note[i])
    if(train_set_note[i] == ".DS_Store"):
      continue
    im = Image.open(os.path.join('/content/drive/My Drive/data/Image_Training_Note/', train_set_note[i]))
    image_train.append(transforms.ToTensor()(im))
print("ACCIDENTAL IMAGE TRAIN SET")
for i in range(1,len(train_set_accidental)):
    print(train_set_accidental[i])
    if(train_set_accidental[i] == ".DS_Store"):
      continue
    im = Image.open(os.path.join('/content/drive/My Drive/data/Image_Training_Accidental/', train_set_accidental[i]))
    image_acc.append(transforms.ToTensor()(im))
    

idx = 0
note_tensor = []
acc_tensor = []
for ret in process_labels_note(label_set):
    note_tensor.append(ret)
for ret in process_labels_acc(label_set_two):
    acc_tensor.append(ret)
'''
for labels in note_tensor:
    print("{}: {}".format("NOTE TENSOR", labels))
for labels in acc_tensor:
    print("{}: {}".format("ACCIDENTAL TENSOR", labels))
for labels in dyn_tensor:
    print("{}: {}".format("DYNAMIC TENSOR", labels))
'''



        
gest_data = Gesture_Dataset_Note_and_Volume(image_train, note_tensor)
acc_data = Gesture_Dataset_Accidental(image_acc, acc_tensor,transform=transforms.ToTensor())
note_loader = torch.utils.data.DataLoader(gest_data, batch_size = btch_sz, shuffle=True, num_workers = 3)
acc_loader = torch.utils.data.DataLoader(acc_data, batch_size = btch_sz,shuffle=True, num_workers = 3)
cnn_note = CNN_Base_Note_and_Volume(540,720).to(device)
cnn_accidental = CNN_Base_Accidental(540,720).to(device)

criterion = nn.MSELoss()
optimizer_note = torch.optim.SGD(cnn_note.parameters(), lr=learning_rate) 
optimizer_acc = torch.optim.SGD(cnn_accidental.parameters(),lr=learning_rate)
optimizer_conv_note = torch.optim.SGD(model_conv.parameters(),lr=learning_rate)
optimizer_conv_acc = torch.optim.SGD(model_conv_2.parameters(),lr=learning_rate)

    
#OPTIMIZATION PROCESS


for n_e in range(epoch):
    for idx, (i,j) in enumerate(note_loader):
        print("Note: Epoch {}, Test {}".format(n_e, idx))
        i=i.to(device)
        #print(i.shape)
        j=j.to(device)
        #print(j.shape)
        print("Phase 1")
        if torch.cuda.is_available():
          op = model_conv(i)
          j = j.float()
          ls = criterion(op,j)
          print("Phase 2")
          optimizer_conv_note.zero_grad()
          ls.backward()
          optimizer_conv_note.step()
          print("Phase 3")
print("NOTE (AND DYNAMIC) TRAINING COMPLETE!")
for n_e in range(epoch):
    for idx, (i,j) in enumerate(acc_loader):
        print("Accidental: Epoch {}, Test {}".format(n_e, idx))
        i=i.to(device)
        j=j.to(device)
        j = j.float()
        print("Phase 1")
        if torch.cuda.is_available():
          op = model_conv_2(i)
          ls = criterion(op,j)
          print("Phase 2")
          optimizer_conv_acc.zero_grad()
          ls.backward()
          optimizer_conv_acc.step()
          print("Phase 3")
print("ACCIDENTAL TRAINING COMPLETE!")


torch.save(model_conv.state_dict(), MODEL_NOTE_PATH)
torch.save(model_conv_2.state_dict(), MODEL_ACC_PATH)
        
print("SAVED!")



cpu
Note: Epoch 0, Test 0
Phase 1
Phase 2
Phase 3
Note: Epoch 0, Test 1
Phase 1


In [8]:
import torch
import torch.nn as nn
import numpy
import pandas
import matplotlib.pyplot as plt
import torch.utils.data as tdata
import os 
from torch import dtype
import pyaudio
import skimage
import pyaudio
from PIL import Image
import itertools

MODEL_NOTE_PATH = '/content/drive/My Drive/data/Current-Model-Note/model_one.txt'
MODEL_ACC_PATH = '/content/drive/My Drive/data/Current-Model-Acc/model_two.txt'
m1 = torch.load(MODEL_NOTE_PATH)
m2 = torch.load(MODEL_ACC_PATH)
print(m1)
print(m2)


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename, quality=1):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      //google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = 1080
      canvas.height = 720
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
from IPython.display import clear_output
idx = 0
def run():
  filename = ''
  while(filename == ''):
    try:
      global idx
      filename = take_photo("image_"+str(idx)+".jpeg")
      if(filename != ''):
        idx += 1
      print('Saved to {}'.format(filename))
      #Show the image which was just taken.
      display(Image(filename))
      clear_output()
    except Exception as err:
      # Errors will be thrown if the user does not have a webcam or if they do not
      # grant the page permission to access it.
      print(str(err))

def run_glob():
    while(True):
      run()

try:
  run_glob()
except Exception as err:
  print("TERMINATING STREAM...")
